<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Feature-selection" data-toc-modified-id="Feature-selection-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Feature selection</a></span></li><li><span><a href="#Imputation" data-toc-modified-id="Imputation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Imputation</a></span></li><li><span><a href="#OneHotEncoding" data-toc-modified-id="OneHotEncoding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>OneHotEncoding</a></span></li><li><span><a href="#Lagging-features" data-toc-modified-id="Lagging-features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Lagging features</a></span></li></ul></div>

In [4]:
import pandas as pd
import numpy as np
import sklearn
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
#path= ''

In [5]:
test=pd.read_csv('test_up.csv')

In [6]:
len(test) #check everything is ok 

506522

In [7]:
test.columns #same

Index(['date', 'store_id', 'parent_chain_id', 'store_country',
       'country_iso_code', 'region_id', 'store_region', 'store_segment',
       'store_activity_name', 'items_first_enabled_date',
       'store_first_saving_date', 'store_last_saving_date', 'item_id',
       'item_name', 'before_price', 'currency_code', 'pickup_start',
       'pickup_end', 'total_supply', 'declared_supply', 'manual_added_supply',
       'manual_removed_supply', 'meals_saved', 'consumer_cancellation',
       'store_cancellation', 'item_price', 'meals_refunded', 'rating_count',
       'sum_rating_overall', 'item_view', 'no_unique_consumers', 'is_enabled',
       'Département', 'index'],
      dtype='object')

# Feature selection 

In [8]:
X_test = test[['date', 'store_id', 'parent_chain_id', 'store_region', 'store_segment',
               'store_first_saving_date', 'before_price', 'total_supply', 'declared_supply', 'manual_added_supply',
               'manual_removed_supply', 'meals_saved', 'consumer_cancellation',
               'store_cancellation', 'item_price', 'meals_refunded', 'rating_count',
               'sum_rating_overall', 'item_view', 'no_unique_consumers',
               'Département']]

# dropna and junked data
X_test = X_test.dropna(subset=['store_first_saving_date'])
X_test['parent_chain_id'] = X_test['parent_chain_id'].fillna(0)
X_test = X_test[X_test['meals_saved'] <= X_test['total_supply']]
print('1', len(X_test))
# create diff saved and diff first
X_test['diff_saved'] = (
    (X_test['total_supply'] - X_test['meals_saved']) / X_test['total_supply']).fillna(0)
X_test['diff_first'] = (pd.to_datetime(X_test['date']) -
                        pd.to_datetime(X_test['store_first_saving_date'])).dt.days
X_test.drop(columns=['store_first_saving_date'], inplace=True)

1 494515


# Imputation

In [9]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer(strategy='median')

impute_median = np.array(X_test['before_price']).reshape(-1, 1)

before_price_was_imputed = X_test['before_price'].isnull()
before_price_imputed = pd.DataFrame(my_imputer.fit_transform(impute_median))

print(len(X_test.columns))
X_test.drop(columns=['before_price'], axis=1, inplace=True)
print('2', len(X_test))
print(len(X_test.columns))

X_test['before_price'] = np.array(before_price_imputed)
print('3', len(X_test))

date = pd.DataFrame()
date['date'] = pd.to_datetime(X_test['date'])
date['dayOfWeek'] = date['date'].dt.day_name()
date_day_string = date['dayOfWeek']
date[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']] = pd.get_dummies(
    date_day_string)[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']]
date['weekend'] = date_day_string.isin(
    ['Saturday', 'Sunday']).replace({True: 1, False: 0})

X_test.drop(columns=['date'], inplace=True)
print(len(X_test))

22
2 494515
21
3 494515
494515


# OneHotEncoding

In [10]:
region_dummies = pd.get_dummies(X_test['store_region'])
segment_dummies = pd.get_dummies(X_test['store_segment'])
X_test = pd.concat([date, X_test, region_dummies, segment_dummies], axis=1)
X_test.drop(columns=['dayOfWeek', 'store_region',
                     'store_segment'], inplace=True)
X_test['Département'] = X_test['Département'].astype('int64')

In [11]:
X_test_final=X_test.copy()


# Lagging features

In [15]:
features_to_lag=['store_id','total_supply', 'meals_saved', 'item_view']

In [16]:
X_temp = X_test_final[features_to_lag]
X_ = np.zeros((len(X_temp), 49))
compteur = 0
for i in tqdm(list(X_temp.store_id.unique())):
    df = X_temp[X_temp['store_id'] == i].copy()
    for i in range(1, 16):
        for col in ['total_supply', 'meals_saved', 'item_view']:
            df["{}_lag_{}".format(col, i)] = df[col].shift(i)
    buf = df.to_numpy()
    X_[compteur:compteur+len(buf), :] = buf
    compteur += len(buf)

In [17]:
X_shifted = pd.DataFrame(X_, columns=df.columns)
X_test_final.drop(columns=['store_id'], axis=1, inplace=True)
X_test_final.reset_index(drop=True, inplace=True)

In [20]:
test = pd.concat([X_test_final, X_shifted], axis=1)

In [ ]:
test.to_csv('final_test_.csv', index=False) #saving df

In [23]:
test

,date,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,weekend,parent_chain_id,...,item_view_lag_12,total_supply_lag_13,meals_saved_lag_13,item_view_lag_13,total_supply_lag_14,meals_saved_lag_14,item_view_lag_14,total_supply_lag_15,meals_saved_lag_15,item_view_lag_15
0,2020-04-29,0,0,1,0,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-30,0,0,0,1,0,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-05-01,0,0,0,0,1,0,0,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-04-28,0,1,0,0,0,0,0,0,4696.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-04-29,0,0,1,0,0,0,0,0,4696.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494510,2020-04-27,1,0,0,0,0,0,0,0,0.0,...,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0
494511,2020-04-28,0,1,0,0,0,0,0,0,0.0,...,5.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0
494512,2020-04-29,0,0,1,0,0,0,0,0,0.0,...,1.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,1.0
494513,2020-04-30,0,0,0,1,0,0,0,0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,4.0
